# Google Earth Engine (GEE) DisALEXI 
## Single Landsat Pre-collection SR Image Example

Example of calling the GEE DisALEXI function on a single Landsat 8 pre-collection LEDAPS at-surface reflectance image.

### Python imports and GEE initialization

In [1]:
import pprint

import ee
from IPython.display import Image

# This should work assuming "eedisalexi" is in the PYTHONPATH
# import eedisalexi.eedisalexi as eedisalexi
# import eedisalexi.landsat as landsat
# import eedisalexi.utils as utils

# This seems super hacky and is not PEP8 but works for now
import os, sys
module_path = os.path.join(os.path.dirname(os.getcwd()), 'eedisalexi')
sys.path.insert(0, module_path)
import eedisalexi
import landsat
import utils

In [2]:
pp = pprint.PrettyPrinter(indent=2)
ee.Initialize()

In [3]:
# AmeriFlux sites adjusted to nearest Landsat cell centroid
# http://sites.ameriflux.lbl.gov/US-NE1/
ne1_xy = [-96.47672812080845, 41.16506126041818]
ne2_xy = [-96.46994024736414, 41.16491226772292]
ne3_xy = [-96.43968912903934, 41.17964494123755]
ne1_geom = ee.Geometry.Point(ne1_xy)
ne2_geom = ee.Geometry.Point(ne2_xy)
ne3_geom = ee.Geometry.Point(ne3_xy)

asset_region = [[-96.99, 41.05], [-96.97, 41.86], [-95.89, 41.83], [-95.92, 41.02]]
test_region = [[-96.5, 41.15], [-96.5, 41.2], [-96.41, 41.2], [-96.41, 41.15]]

In [4]:
ssebop_et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
ndvi_palette = '#EFE7E1,#003300'

### Construct a TOA like image

Combine the the LEDAPS SR shortwave bands and the TOA thermal and BQA bands

In [5]:
sr_img = ee.Image('LANDSAT/LC8_SR/LC80280312014189').multiply(0.0001)
toa_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_028031_20140708')
landsat_img = ee.Image([
    sr_img.select(['B2']), sr_img.select(['B3']),
    sr_img.select(['B4']), sr_img.select(['B5']),
    sr_img.select(['B6']), sr_img.select(['B7']),
    toa_img.select('B10'), toa_img.select('BQA')
])
# Copy properties from the TOA image
landsat_img = ee.Image(landsat_img.copyProperties(toa_img, [
    'system:index', 'system:time_start', 'SPACECRAFT_ID',
    'K1_CONSTANT_BAND_10', 'K2_CONSTANT_BAND_10']))
# pp.pprint(landsat_img.getInfo())

In [6]:
# Extract geotransform and spatial reference
image_crs = landsat_img.select('B2').projection().crs().getInfo()
image_transform = landsat_img.select('B2').projection().getInfo()['transform']
image_geom = landsat_img.select('B2').geometry()
print(image_crs)
print(image_transform)

EPSG:32614
[30.0, 0.0, 632385.0, 0.0, -30.0, 4743015.0]


In [7]:
validation_geom = ee.Image('users/cgmorton/disalexi/LC08_028031_20140708/albedo') \
    .geometry()

    # {}'min': [0.01, 0.01, 0.01], 'max': [0.4, 0.4, 0.4], 'gamma': [1.8, 1.8, 1.8]}
vis_image = landsat_img \
    .visualize(bands=['B4', 'B3', 'B2'], min=[0.01, 0.01, 0.01], max=[0.3, 0.3, 0.3]) \
    .reproject(crs=image_crs, crsTransform=image_transform) \
    .paint(validation_geom, color=0.5, width=1) \
    .clip(image_geom)
Image(url=vis_image.getThumbURL())
# print(vis_image.getThumbURL())

### Prep the raw Landsat TOA image for use in GEE DisALEXI

The prep function will rename the bands to generic names, compute albedo, ndvi, lai, and extract the fmask image from the BQA band.

In [8]:
input_img = ee.Image(landsat.Landsat(landsat_img).prep())
mask_img = input_img.select('cfmask').eq(0)
# pp.pprint(input_img.getInfo())

Show the NDVI band for the validation asset area

In [9]:
vis_img = input_img.select(['ndvi']).updateMask(mask_img) \
    .paint(ee.Geometry.Polygon(test_region), 'red', 1)
Image(url=vis_img.getThumbURL({
    'region': asset_region, 'min':-0.1, 'max':0.9, 'palette': ndvi_palette}))

Show the NDVI band for the study area

In [10]:
Image(url=input_img.select(['ndvi']).updateMask(mask_img).getThumbURL({
    'region': test_region, 'min':-0.1, 'max':0.9, 'palette': ndvi_palette}))

### Extract input values at the test point

In [11]:
print('LAI:    {:12.8f}'.format(float(
    utils.image_value(input_img, xy=ne1_xy)['lai'])))
print('NDVI:   {:12.8f}'.format(float(
    utils.image_value(input_img, xy=ne1_xy)['ndvi'])))
print('Albedo: {:12.8f}'.format(float(
    utils.image_value(input_img, xy=ne1_xy)['albedo'])))
print('LST:    {:12.8f} [K]'.format(float(
    utils.image_value(input_img, xy=ne1_xy)['lst'])))

LAI:      4.60945286
NDVI:     0.86999571
Albedo:   0.19520390
LST:    300.37899609 [K]


### Initialize the DisALEXI object

In [12]:
# Initialize the DisALEXI object using all default values
# NLCD Landcover, NED Elevation, CONUS ALEXI ET
disalexi = eedisalexi.DisALEXI(input_img, iterations=10)

### Extract the ancillary values at the test point

In [13]:
disalexi._set_alexi_et_vars()
disalexi._set_elevation_vars()
disalexi._set_landcover_vars()
disalexi._set_solar_vars()
disalexi._set_weather_vars()
print('ALEXI ET:   {:>12.8f} [mm]'.format(float(utils.image_value(
    disalexi.alexi_et, xy=ne1_xy)['alexi_et'])))
print('Elevation:  {:>12.8f} [m]'.format(float(utils.image_value(
    disalexi.elevation, xy=ne1_xy)['elevation'])))
print('Rs Hourly:  {:>12.8f} [W m-2]'.format(float(utils.image_value(
    disalexi.rs1, xy=ne1_xy)['rs'])))
print('Rs Daily:  {:>13.8f} [W m-2]'.format(float(utils.image_value(
    disalexi.rs24, xy=ne1_xy)['rs'])))
print('Wind Speed: {:>12.8f} [m/s]'.format(float(utils.image_value(
    disalexi.windspeed, xy=ne1_xy)['windspeed'])))
print('Land Cover: {:3}'.format(int(utils.image_value(
    disalexi.landcover, xy=ne1_xy)['landcover'])))

ALEXI ET:     6.34297943 [mm]
Elevation:  356.74499512 [m]
Rs Hourly:  925.85181250 [W m-2]
Rs Daily:  8506.97167969 [W m-2]
Wind Speed:   4.11969798 [m/s]
Land Cover:  82


### Compute Air Temperature

In [14]:
image_transform = landsat_img.select('B2').projection().getInfo()['transform']
# Reduce the computation resoultion
# image_transform = [120.0, 0.0, 632685.0, 0.0, -120.0, 4742715.0]

In [15]:
t_air_img = ee.Image(disalexi.compute_ta()) \
    .reproject(crs=image_crs, crsTransform=image_transform)

In [16]:
# print('Output Ta:\n{}'.format(
#     pprint.pformat(utils.image_value(t_air_img)['t_air'])))

In [17]:
# t_air_url = t_air_img.getThumbURL({
#     'region': test_region, 'min':280, 'max':310})
# print(t_air_url)

In [18]:
# Image(url=t_air_url)

### Aggregate Air Temperature to ALEXI ET cells

In [19]:
t_air_coarse_img = disalexi.aggregate(t_air_img)

In [20]:
# print('Coarse Ta: {}'.format(pprint.pformat(
#     utils.image_value(t_air_coarse_img, xy=ne1_xy, scale=1)['t_air'])))

Coarse Ta: 296.65529296395783


In [21]:
# t_air_coarse_url = t_air_coarse_img.getThumbUrl({
#     'region': test_region, 'min':280, 'max':310})
# print(t_air_coarse_url)

In [22]:
# Image(url=t_air_coarse_url)

### Compute ET

In [23]:
et_img = disalexi.compute_et(t_air_coarse_img) \
    .reproject(crs=image_crs, crsTransform=image_transform)

In [24]:
# print('ET: {}'.format(pprint.pformat(
#     utils.image_value(et_img, xy=ne1_xy)['et'])))

ET: 7.178786715171644


In [25]:
# et_url = et_img.getThumbURL({
#     'region': test_region, 'min': 0, 'max': 8, 
#     'palette': ','.join(ssebop_et_palette)})
# print(et_url)

In [26]:
# Image(url=et_url)

### Aggregate ET to ALEXI ET cells

In [27]:
et_coarse_img = disalexi.aggregate(et_img)

In [28]:
# print('Coarse ET: {}'.format(pprint.pformat(
#     utils.image_value(et_coarse_img, xy=ne1_xy, scale=1)['et'])))

### Exports

In [29]:
tair_task = ee.batch.Export.image.toDrive(
    t_air_img,
    description='LC08_028031_20140708_pre_sr_tair',
    folder='DisALEXI',
    fileNamePrefix='LC08_028031_20140708_pre_sr_tair',
    dimensions='3000x3000',
    crs=image_crs,
    crsTransform=image_transform,
    maxPixels=1E9)

In [30]:
tair_coarse_task = ee.batch.Export.image.toDrive(
    t_air_coarse_img,
    description='LC08_028031_20140708_pre_sr_tair_coarse',
    folder='DisALEXI',
    fileNamePrefix='LC08_028031_20140708_pre_sr_tair_coarse',
    dimensions='3000x3000',
    crs=image_crs,
    crsTransform=image_transform,
    maxPixels=1E9)

In [31]:
et_task = ee.batch.Export.image.toDrive(
    et_img,
    description='LC08_028031_20140708_pre_sr_et',
    folder='DisALEXI',
    fileNamePrefix='LC08_028031_20140708_pre_sr_et',
    dimensions='3000x3000',
    crs=image_crs,
    crsTransform=image_transform,
    maxPixels=1E9)

In [32]:
et_coarse_task = ee.batch.Export.image.toDrive(
    et_coarse_img,
    description='LC08_028031_20140708_pre_sr_et_coarse',
    folder='DisALEXI',
    fileNamePrefix='LC08_028031_20140708_pre_sr_et_coarse',
    dimensions='3000x3000',
    crs=image_crs,
    crsTransform=image_transform,
    maxPixels=1E9)

In [33]:
# tair_task.start()
# tair_coarse_task.start()
# et_task.start()
# et_coarse_task.start()
# print('Tair Status:        {}'.format(tair_task.status()))
# print('Tair Coarse Status: {}'.format(tair_coarse_task.status()))
# print('ET Status:          {}'.format(et_task.status()))
# print('ET Status:          {}'.format(et_coarse_task.status()))

Tair Coarse Status: {'id': 'ZZF6WZZ7X2FHW6TIFZ2PSK7Q', 'state': 'READY', 'creation_timestamp_ms': 1507393741124, 'update_timestamp_ms': 1507393741124, 'description': 'LC08_028031_20140708_pre_sr_tair_coarse', 'task_type': 'EXPORT_IMAGE'}
